# DNN-Detection-keras

## 实验目的

探究`DNN`在`OFMD`通信系统中进行信道估计和信号检测的能力

## 实验内容

将`DNN`训练模型的结构与传统算法进行比较，以及探究导频长度、激活函数、学习率等参数对性能的影响

## 实验平台

- `OS`: `Manjaro Linux x86_64`
- `Kernel`: `5.4.80-1-MANJARO`
- `Python`: `3.8.6`
- `Jupyter Notebook`: `6.1.4`
- `Matplotlib`: `3.3.3`
- `Numpy`: `1.19.4`
- `Tensorflow`: `2.3.1`
- `Keras`: `2.4.0`

## 实验步骤

运行`run_all`函数，修改其参数值，分别是`CP`, `P`导频长度, `activation`激活函数, `lr`学习率，得到的结果和数据保存在`reslut`文件夹中，其中文件名后缀格式为`[CP]-[P]-[activation]-[lr]`

### 程序的头文件

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from generations import *


### 创建神经网络模型

In [ ]:
def init_model(activation):
    model = models.Sequential()
    model.add(
        layers.Dense(
            n_hidden_1, activation=activation, input_shape=(payloadBits_per_OFDM * 2,)
        )
    )
    model.add(layers.Dense(n_hidden_2, activation=activation))
    model.add(layers.Dense(n_hidden_3, activation=activation))
    model.add(layers.Dense(n_output, activation="sigmoid"))

    return model


### 配置优化器

In [ ]:
def config_optimizer(model, lr):
    model.compile(
        optimizer=optimizers.RMSprop(lr=lr), loss="mse", metrics=[bit_err]
    )

    return model


### 训练模型

In [ ]:
def train_model(CP, P, pilotCarriers, dataCarriers, pilotValue, model):
    history = model.fit(
        training_gen(CP, P, pilotCarriers, dataCarriers, pilotValue, 1000, 20),
        steps_per_epoch=250,
        epochs=100,
        validation_data=validation_gen(
            CP, P, pilotCarriers, dataCarriers, pilotValue, 1000, 20
        ),
        validation_steps=1,
        verbose=2,
    )

    return model, history


### 获得相应的数据，以及保存数据

In [ ]:
def save_result(history, result_npz):
    loss = history.history["loss"]
    bit_err = history.history["bit_err"]
    val_loss = history.history["val_loss"]
    val_bit_err = history.history["val_bit_err"]

    np.savez(
        result_npz,
        loss=loss,
        bit_err=bit_err,
        val_loss=val_loss,
        val_bit_err=val_bit_err,
    )


### 验证经训练得到的模型在各个SNR下的BER性能

In [ ]:
def save_evaluate(
    CP, P, pilotCarriers, dataCarriers, pilotValue, model, result_npy
):
    BER = []
    for SNR in range(5, 30, 2):
        y = model.evaluate(
            validation_gen(
                CP, P, pilotCarriers, dataCarriers, pilotValue, 10000, SNR
            ),
            steps=1,
        )  # y=[loss_value, metrics_value]
        BER.append(y[1])

    np.save(result_npy, BER)


### 绘制训练ber和验证ber

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

def draw_training_ber(result_npz, suffix):
    datas = np.load(result_npz)
    loss = datas["loss"]
    bit_err = datas["bit_err"]
    val_loss = datas["val_loss"]
    val_bit_err = datas["val_bit_err"]

    epochs = range(1, len(loss) + 1)

    plt.semilogy(epochs, bit_err, "y", label="Training bit_err")
    plt.semilogy(epochs, val_bit_err, "b", label="Validation bit_err")
    plt.xlabel("epochs")
    plt.ylabel("BER")
    plt.title("Training and validation bit_err")
    plt.legend()
    plt.savefig(os.path.join("result", "bit_err" + suffix + ".jpg"))

    plt.figure()

    return epochs, loss, val_loss


### 绘制训练损失和验证损失

In [ ]:
def draw_loss(epochs, loss, val_loss, suffix):
    plt.semilogy(epochs, loss, "y", label="Training loss")
    plt.semilogy(epochs, val_loss, "b", label="Validation loss")
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.title("Training and validation loss")
    plt.legend()
    plt.savefig(os.path.join("result", "loss" + suffix + ".jpg"))
    
    plt.figure()


### 绘制基于深度学习的信号检测方法的BER性能

In [ ]:
def draw_learning_ber(result_npy, suffix):
    BER = np.load(result_npy)
    SNR = range(5, 30, 2)

    plt.semilogy(SNR, BER, "b", label="BER")
    plt.xlabel("SNR(dB)")
    plt.ylabel("BER")
    plt.title("BER of deep learning based approach")
    plt.legend()
    plt.savefig(os.path.join("result", "BER" + suffix + ".jpg"))

    plt.show()


In [ ]:
def run_all(CP, P, activation, lr):
    suffix = "_" + "-".join([str(CP), str(P), activation, str(lr)])
    result_npz = os.path.join("result", "result" + suffix + ".npz")
    result_npy = "BER" + suffix + ".npy"

    pilotCarriers, dataCarriers = get_carriers(P)
    pilotValue = get_pilot_value(P)

    model = init_model(activation)
    model = config_optimizer(model, lr)
    model, history = train_model(
        CP, P, pilotCarriers, dataCarriers, pilotValue, model
    )

    save_result(history, result_npz)
    save_evaluate(
        CP, P, pilotCarriers, dataCarriers, pilotValue, model, result_npy
    )

    epochs, loss, val_loss = draw_training_ber(result_npz, suffix)
    draw_loss(epochs, loss, val_loss, suffix)
    draw_learning_ber(result_npy, suffix)


`CP` = 16, `P` = 64, `activation` = relu, `lr` = 0.001

In [ ]:

run_all(16, 64, "relu", 0.001)


`CP` = 16, `P` = 8, `activation` = relu, `lr` = 0.001

In [ ]:
run_all(16, 8, "relu", 0.001)

`CP` = 0, `P` = 64, `activation` = relu, `lr` = 0.001

In [ ]:
run_all(0, 64, "relu", 0.001)

`CP` = 0, `P` = 8, `activation` = relu, `lr` = 0.001

In [ ]:
run_all(0, 8, "relu", 0.001)

`CP` = 16, `P` = 64, `activation` = relu, `lr` = 0.004

In [ ]:
run_all(16, 64, "relu", 0.004)

`CP` = 16, `P` = 8, `activation` = relu, `lr` = 0.004

In [ ]:
run_all(16, 8, "relu", 0.004)

`CP` = 16, `P` = 64, `activation` = sigmoid, `lr` = 0.001

In [ ]:
run_all(16, 64, "sigmoid", 0.001)

`CP` = 16, `P` = 8, `activation` = sigmoid, `lr` = 0.001

In [ ]:
run_all(16, 8, "sigmoid", 0.001)

## 实验总结

- `P`的影响
  - `P`较大时，性能类似
  - `P`较小时，基于`DNN`的性能更好
- `CP`的影响
  - 可以克服多径干扰，需要额外的能量开销
  - 传统接收机受`CP`的影响较大， 没有`CP`，基于`DNN`的接收机有更大的增益
- 学习率的影响
  - 将学习率从调整为`0.001`调整为`0.004`，波动变大，在训练`15`次左右之后，性能随着训练次数的增加越来越差
- 激活函数的影响
  - 将激活函数从调整为`relu`调整为`sigmoid`，训练增益变小，波动更大
